# deep learning implementation for CF
as shown in this paper 

J. Rappaz, J. McAuley, and K. Aberer, "Recommendation on Live-Streaming Platforms: Dynamic Availability and Repeat Consumption," EPFL, Lausanne, Switzerland, and University of California, San Diego, CA, USA, 2021. [Online]. Available: https://cseweb.ucsd.edu/~jmcauley/pdfs/recsys21b.pdf. [Accessed: Dec. 23, 2023].

we can see that although the SVD approach can recommend streamers to the user it cannot recommend streams due to their reliance on timing as a user cannot consume a live stream before or after it's release. we will use deep learning to try and enhance our recommendations by including the timing of streams and by making sure there is a time variable for when a user logs in and consumes items. For this model our items will be changed to streams in order to make this possible.

## What we will try and advance the model with


streamers are not constantly broadcasting so at the time a user is logged in they are exposed to only items they can view at that time. This means that our positive interactions stay positive however if an item is not avaialable when the user logs in that should not be made a negative interaction. [1] a model should only learn from valid options that are available. 

as we are modelling temporal data a flat architecture such as an MLP is unsuitable as it will flatten the input and lose any sequencing of the data. we are going to treat the problem as a sequential problem here we will use an attention model similar to the SASRec model shown in this paper:

W.-C. Kang and J. McAuley, "Self-Attentive Sequential Recommendation," arXiv, 2018. [Online]. Available: https://arxiv.org/pdf/1808.09781.pdf. [Accessed: Dec. 23, 2023].



### creating our model inputs
the attention model requires sequences of user interactions ordered by time. These sequencces need to all be of the same length this can be achieved by padding with null values if the sequence is too short and removing older items if the sequence is too long 

In [1]:
# importing the data
import pandas as pd
import os 
import numpy as np
from sklearn.preprocessing import MinMaxScaler


file_path = os.path.join(os.getcwd(),'Datasets/100k_a.csv')
cols = ["user","stream","streamer","start","stop"]
data = pd.read_csv(file_path, header=None, names=cols)
data.user = pd.factorize(data.user)[0]+1
data['streamer_raw'] = data.streamer
data.streamer = pd.factorize(data.streamer)[0]+1
data[['start', 'stop']] = MinMaxScaler().fit_transform(data[['start', 'stop']])
print("Num users: ", data.user.nunique())
print("Num streamers: ", data.streamer.nunique())
print("Num interactions: ", len(data))


/Users/curranmaguire/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Num users:  100000
Num streamers:  162625
Num interactions:  3051733


I will need to show graphs that explain why the timing of my data matters 


# initial model

let's first create an LSTM recommender that will help us recommend items by taking into account the time of consuption of items. LSTM's are good for sequential data as they are a type of RNN. 

### data preperation
first we will need to generate sequences of user input these will need to be orgonized by the time of interaction

In [2]:

def generate_sequences(df, sequence_length):
    df_sorted = df.sort_values(by=['user', 'start'])
    grouped = df_sorted.groupby('user')
    sequences = []
    labels = []
    seq_time = []
    for user, group in grouped:
        interactions = [x for x in group['stream'].values]
        #interactions_time = [tuple(x) for x in group[['stream','start']].values]#allows us to create sequences that can takeinto account the start time for availability
        seq = interactions[-sequence_length:-1]
        if len(seq) < sequence_length:
            for i in range(sequence_length- len(seq)):
                seq.insert(0,0)
        if len(seq) > sequence_length:
            for i in range(len(seq) - sequence_length):
                seq.pop(0)
        labels.append(interactions[-1])#data target as last item in the list
        sequences.append(seq)
        return sequences, labels
    

X,y = generate_sequences(data, 10)
np.save('Datasets/X.data', X)
np.save('Datasets/y.data', y)

### creating the LSTM class 

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping


class LSTM_model:
    def __init__(self, n_features, units, dropout_rate, recurrent_dropout, series_len, prediction_len):
        self.n_in = series_len
        self.n_out = prediction_len
        self.n_features = n_features
        self.units = units
        self.dropout_rate = dropout_rate
        self.recurrent_dropout = recurrent_dropout
        self.model = self._build_model()
    def _build_model(self):
        model = Sequential()
        model.add(LSTM(self.units, activation='relu', input_shape=(self.n_in, self.n_features),
                       dropout=self.dropout_rate, recurrent_dropout=self.recurrent_dropout))
        model.add(RepeatVector(self.n_out))
        model.add(LSTM(self.units, activation='relu', return_sequences=True, 
                       dropout=self.dropout_rate, recurrent_dropout=self.recurrent_dropout))
        model.add(TimeDistributed(Dense(self.n_features)))
        model.compile(optimizer='adam', loss='mse')
        return model
    
    def train(self, X,y, batch_size, epochs, validation_split = 0.3):
        es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=5, restore_best_weights=True)
        self.model.fit(X, y, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=validation_split, callbacks=[es])

    def predict(self, X):
        return self.model.predict(X)


: 